In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 设置回测参数
start_date = '2018-01-01'
end_date = '2025-01-01'
tickers = ['QQQ', 'SPY', 'SHY', 'SOYB', 'GLD']
initial_capital = 10000
n_selected = 2  # 每季度选择前2名

# 下载数据
data = yf.download(tickers, start=start_date, end=end_date)['Close']

# 计算季度收益率
quarterly_returns = data.resample('Q').last().pct_change()

# 创建交易信号DataFrame
signals = pd.DataFrame(index=data.index, columns=data.columns)

# 生成轮动信号
for i in range(1, len(quarterly_returns)):
    # 获取上一季度收益率排名
    prev_quarter = quarterly_returns.iloc[i-1]
    top_assets = prev_quarter.nlargest(n_selected).index
    
    # 确定当前季度的日期范围
    quarter_start = quarterly_returns.index[i-1]
    quarter_end = quarterly_returns.index[i] if i < len(quarterly_returns) else data.index[-1]
    
    # 为当前季度分配信号
    for asset in data.columns:
        if asset in top_assets:
            signals.loc[(signals.index > quarter_start) & (signals.index <= quarter_end), asset] = 1/n_selected
        else:
            signals.loc[(signals.index > quarter_start) & (signals.index <= quarter_end), asset] = 0

# 前向填充信号
signals = signals.fillna(method='ffill')

# 计算每日收益率
daily_returns = data.pct_change()
strategy_returns = (signals.shift(1) * daily_returns).sum(axis=1)

# 计算累计收益
cumulative_returns = (1 + strategy_returns).cumprod()
portfolio_value = initial_capital * cumulative_returns

# 计算年化收益率
total_days = (data.index[-1] - data.index[0]).days
annualized_return = (portfolio_value[-1] / initial_capital) ** (365/total_days) - 1

# 计算年化波动率
annualized_volatility = strategy_returns.std() * np.sqrt(252)

# 计算夏普比率 (假设无风险利率为0)
sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)

# 计算Alpha (以SPY为基准)
benchmark_returns = data['SPY'].pct_change().dropna()
strategy_returns_aligned = strategy_returns[strategy_returns.index.isin(benchmark_returns.index)]
covariance = np.cov(strategy_returns_aligned, benchmark_returns)[0, 1]
variance = benchmark_returns.var()
beta = covariance / variance
alpha = (annualized_return - (strategy_returns.mean() * 252 - beta * benchmark_returns.mean() * 252))

# 计算每年年化收益率
yearly_returns = portfolio_value.resample('Y').last().pct_change()
yearly_returns.index = yearly_returns.index.year
yearly_returns = yearly_returns.dropna()

# 打印结果
print(f"回测期间: {start_date} 至 {end_date}")
print(f"初始资金: ${initial_capital:,.2f}")
print(f"最终资金: ${portfolio_value[-1]:,.2f}")
print(f"总年化收益率: {annualized_return*100:.2f}%")
print(f"年化波动率: {annualized_volatility*100:.2f}%")
print(f"夏普比率: {sharpe_ratio:.2f}")
print(f"Alpha: {alpha*100:.2f}%")
print("\n每年年化收益率:")
print((yearly_returns * 100).round(2).astype(str) + '%')

/tmp/ipykernel_6844/1408347554.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)['Close']
[*********************100%***********************]  5 of 5 completed

回测期间: 2018-01-01 至 2025-01-01
初始资金: $10,000.00
最终资金: $18,447.09
总年化收益率: 9.14%
年化波动率: 17.08%
夏普比率: 0.60
Alpha: 8.04%

每年年化收益率:
Date
2019    13.38%
2020    11.08%
2021    25.46%
2022     -6.3%
2023      6.3%
2024    26.46%
dtype: object



/tmp/ipykernel_6844/1408347554.py:17: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quarterly_returns = data.resample('Q').last().pct_change()
/tmp/ipykernel_6844/1408347554.py:40: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  signals = signals.fillna(method='ffill')
/tmp/ipykernel_6844/1408347554.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  signals = signals.fillna(method='ffill')
/tmp/ipykernel_6844/1408347554.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 设置回测参数
start_date = '2018-01-01'
end_date = '2024-12-31'
tickers = ['QQQ', 'SPY', 'SHY', 'SOYB', 'GLD']
initial_capital = 10000
n_selected = 2  # 每季度选择前2名

# 下载数据
data = yf.download(tickers, start=start_date, end=end_date)['Close']

# 计算季度收益率
quarterly_returns = data.resample('Q').last().pct_change()

# 创建交易信号DataFrame
signals = pd.DataFrame(index=data.index, columns=data.columns)

# 生成轮动信号
for i in range(1, len(quarterly_returns)):
    # 获取上一季度收益率排名
    prev_quarter = quarterly_returns.iloc[i-1]
    top_assets = prev_quarter.nlargest(n_selected).index
    
    # 确定当前季度的日期范围
    quarter_start = quarterly_returns.index[i-1]
    quarter_end = quarterly_returns.index[i] if i < len(quarterly_returns) else data.index[-1]
    
    # 为当前季度分配信号
    for asset in data.columns:
        if asset in top_assets:
            signals.loc[(signals.index > quarter_start) & (signals.index <= quarter_end), asset] = 1/n_selected
        else:
            signals.loc[(signals.index > quarter_start) & (signals.index <= quarter_end), asset] = 0

# 前向填充信号
signals = signals.fillna(method='ffill')

# 计算每日收益率
daily_returns = data.pct_change()
strategy_returns = (signals.shift(1) * daily_returns).sum(axis=1)

# 计算累计收益
cumulative_returns = (1 + strategy_returns).cumprod()
portfolio_value = initial_capital * cumulative_returns

# 计算年化收益率
total_days = (data.index[-1] - data.index[0]).days
annualized_return = (portfolio_value[-1] / initial_capital) ** (365/total_days) - 1

# 计算年化波动率
annualized_volatility = strategy_returns.std() * np.sqrt(252)

# 计算夏普比率 (假设无风险利率为0)
sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)

# 计算Alpha (以SPY为基准)
benchmark_returns = data['SPY'].pct_change().dropna()
strategy_returns_aligned = strategy_returns[strategy_returns.index.isin(benchmark_returns.index)]
covariance = np.cov(strategy_returns_aligned, benchmark_returns)[0, 1]
variance = benchmark_returns.var()
beta = covariance / variance
alpha = (annualized_return - (strategy_returns.mean() * 252 - beta * benchmark_returns.mean() * 252))

# 计算最大回撤
rolling_max = portfolio_value.cummax()
daily_drawdown = (portfolio_value - rolling_max) / rolling_max
max_drawdown = daily_drawdown.min()

# 计算每年年化收益率
yearly_returns = portfolio_value.resample('Y').last().pct_change()
yearly_returns.index = yearly_returns.index.year
yearly_returns = yearly_returns.dropna()

# 计算每半年年化收益率
halfyear_returns = portfolio_value.resample('6M').last().pct_change()
halfyear_returns.index = halfyear_returns.index.strftime('%Y-%m')
halfyear_returns = halfyear_returns.dropna()

# 打印结果
print(f"回测期间: {start_date} 至 {end_date}")
print(f"初始资金: ${initial_capital:,.2f}")
print(f"最终资金: ${portfolio_value[-1]:,.2f}")
print(f"总年化收益率: {annualized_return*100:.2f}%")
print(f"年化波动率: {annualized_volatility*100:.2f}%")
print(f"夏普比率: {sharpe_ratio:.2f}")
print(f"Alpha: {alpha*100:.2f}%")
print(f"最大回撤: {max_drawdown*100:.2f}%")
print("\n每年年化收益率:")
print((yearly_returns * 100).round(2).astype(str) + '%')
print("\n每半年年化收益率:")
print((halfyear_returns * 100).round(2).astype(str) + '%')

/tmp/ipykernel_6844/3370447933.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)['Close']
[*********************100%***********************]  5 of 5 completed

回测期间: 2018-01-01 至 2024-12-31
初始资金: $10,000.00
最终资金: $18,423.18
总年化收益率: 9.12%
年化波动率: 17.09%
夏普比率: 0.60
Alpha: 8.07%
最大回撤: -30.86%

每年年化收益率:
Date
2019    13.38%
2020    11.08%
2021    25.46%
2022     -6.3%
2023      6.3%
2024     26.3%
dtype: object

每半年年化收益率:
Date
2018-07      4.33%
2019-01     -9.12%
2019-07      2.13%
2020-01      9.63%
2020-07    -10.46%
2021-01      25.3%
2021-07     15.22%
2022-01     -0.77%
2022-07     -0.78%
2023-01      3.52%
2023-07       9.8%
2024-01     -3.19%
2024-07     16.43%
2025-01      6.42%
dtype: object



/tmp/ipykernel_6844/3370447933.py:17: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quarterly_returns = data.resample('Q').last().pct_change()
/tmp/ipykernel_6844/3370447933.py:40: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  signals = signals.fillna(method='ffill')
/tmp/ipykernel_6844/3370447933.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  signals = signals.fillna(method='ffill')
/tmp/ipykernel_6844/3370447933.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by

In [1]:
pip install akshare --upgrade

  Using cached akshare-1.17.25-py3-none-any.whl (1.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 107.1 kB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:010m
  Using cached jsonpath-0.82.2.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 3.6 MB/s eta 0:00:00m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 319.2 kB/s eta 0:00:001m302.3 kB/s eta 0:00:01
  Using cached xlrd-2.0.2-py2.py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 KB 1.4 MB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.1/326.1 KB 479.3 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import akshare as ak
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 设置中文字体 - 使用Ubuntu默认字体
try:
    # 尝试获取Ubuntu默认中文字体
    font_path = "/usr/share/fonts/truetype/wqy/wqy-microhei.ttc"  # Ubuntu常见中文字体路径
    font_prop = fm.FontProperties(fname=font_path)
    plt.rcParams['font.family'] = font_prop.get_name()
except:
    # 如果失败，使用系统默认字体
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置默认字体为黑体
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 获取当前日期和5年前的日期
end_date = datetime.now().strftime('%Y%m%d')
start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y%m%d')

# 获取创业板ETF(159525.SZ)历史数据
def get_etf_data(symbol, start_date, end_date):
    try:
        # 尝试使用AkShare获取数据
        df = ak.stock_zh_a_hist(symbol=symbol, period="daily", 
                               start_date=start_date, end_date=end_date, 
                               adjust="hfq")
        if df.empty:
            raise ValueError("获取的数据为空")
        return df
    except Exception as e:
        print(f"使用AkShare获取数据失败: {e}")
        print("尝试使用备用方法...")
        # 备用方法：使用yfinance
        import yfinance as yf
        symbol_yf = symbol.split('.')[0] + '.SZ'
        df = yf.download(symbol_yf, start=start_date[:4]+'-'+start_date[4:6]+'-'+start_date[6:8],
                        end=end_date[:4]+'-'+end_date[4:6]+'-'+end_date[6:8])
        df = df.reset_index()
        df.rename(columns={'Date': '日期', 'Close': '收盘', 'Open': '开盘',
                          'High': '最高', 'Low': '最低', 'Volume': '成交量'}, inplace=True)
        return df

# 获取数据
symbol = "588870.SH"
df = get_etf_data(symbol, start_date, end_date)

# 检查数据是否获取成功
if df.empty:
    raise ValueError("未能获取到有效数据，请检查股票代码或网络连接")

# 数据处理
df['日期'] = pd.to_datetime(df['日期'])
df.set_index('日期', inplace=True)
df = df.sort_index()

# 计算收益率
initial_price = df['收盘'].iloc[0]
df['累计收益率'] = (df['收盘'] / initial_price - 1) * 100  # 百分比表示

# 计算年化收益率
total_days = (df.index[-1] - df.index[0]).days
annualized_return = (df['收盘'].iloc[-1] / initial_price) ** (365/total_days) - 1

# 计算最大回撤
rolling_max = df['收盘'].cummax()
daily_drawdown = (df['收盘'] - rolling_max) / rolling_max
max_drawdown = daily_drawdown.min()

# 绘制收益率曲线图
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['累计收益率'], label='累计收益率(%)', color='#1f77b4')
plt.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)

# 标记关键点
max_return_date = df['累计收益率'].idxmax()
max_return_value = df['累计收益率'].max()
plt.scatter(max_return_date, max_return_value, color='red', 
           label=f'最高收益: {max_return_value:.2f}%')

current_return = df['累计收益率'].iloc[-1]
plt.scatter(df.index[-1], current_return, color='green', 
           label=f'当前收益: {current_return:.2f}%')

# 添加标题和标签
plt.title(f'创业板ETF({symbol}.SZ) 近5年收益率曲线\n'
         f'年化收益率: {annualized_return*100:.2f}% | 最大回撤: {max_drawdown*100:.2f}%',
         fontsize=14)
plt.xlabel('日期')
plt.ylabel('累计收益率(%)')
plt.legend(loc='best')
plt.grid(True, linestyle='--', alpha=0.6)

# 优化x轴日期显示
plt.gcf().autofmt_xdate()

plt.tight_layout()
plt.show()

# 打印关键指标
print(f"\n{symbol}.SZ 近5年表现分析:")
print(f"起始日期: {df.index[0].strftime('%Y-%m-%d')}")
print(f"结束日期: {df.index[-1].strftime('%Y-%m-%d')}")
print(f"起始价格: {initial_price:.2f}")
print(f"结束价格: {df['收盘'].iloc[-1]:.2f}")
print(f"累计收益率: {df['累计收益率'].iloc[-1]:.2f}%")
print(f"年化收益率: {annualized_return*100:.2f}%")
print(f"最大回撤: {max_drawdown*100:.2f}%")

使用AkShare获取数据失败: 获取的数据为空
尝试使用备用方法...


/tmp/ipykernel_9489/994266202.py:39: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol_yf, start=start_date[:4]+'-'+start_date[4:6]+'-'+start_date[6:8],
HTTP Error 404: 
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['588870.SZ']: YFTzMissingError('possibly delisted; no timezone found')


ValueError: 未能获取到有效数据，请检查股票代码或网络连接

In [3]:
import akshare as ak
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Configuration
symbol = "588000"
market = ""
years_back = 5
initial_capital = 10000  # USD

# Calculate date range
end_date = datetime.now().strftime("%Y%m%d")
start_date = (datetime.now() - timedelta(days=365*years_back)).strftime("%Y%m%d")

try:
    # Attempt to fetch data using AkShare
    print(f"Attempting to fetch data for {symbol} from {start_date} to {end_date}")
    
    # Try different AkShare functions for ETF data
    df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=start_date, end_date=end_date)
    
    if df.empty:
        raise ValueError("No data returned from AkShare")
    
    # Process data
    df['date'] = pd.to_datetime(df['日期'])  # Chinese column name for date
    df.set_index('date', inplace=True)
    df['close'] = df['收盘']  # Chinese column name for close price
    df = df[['close']]
    
    # Calculate returns
    df['daily_return'] = df['close'].pct_change()
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    df['portfolio_value'] = initial_capital * df['cum_return']
    
    # Calculate performance metrics
    total_return = (df['portfolio_value'].iloc[-1] / initial_capital - 1) * 100
    annualized_return = ((1 + total_return/100) ** (1/years_back) - 1) * 100
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['portfolio_value'], label=f'{symbol} Performance')
    plt.title(f'{symbol} Backtest Results ({start_date} to {end_date})')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Print performance summary
    print("\nBacktest Results:")
    print(f"Symbol: {symbol}.{market}")
    print(f"Period: {start_date} to {end_date}")
    print(f"Initial Capital: ${initial_capital:,.2f}")
    print(f"Final Value: ${df['portfolio_value'].iloc[-1]:,.2f}")
    print(f"Total Return: {total_return:.2f}%")
    print(f"Annualized Return: {annualized_return:.2f}%")
    
except Exception as e:
    print(f"\nError: Could not fetch data for {symbol}.{market} using AkShare")
    print(f"Reason: {str(e)}")
    print("\nPossible solutions:")
    print("1. Check if the symbol is correct and exists")
    print("2. Try alternative data sources (Yahoo Finance, Tushare, etc.)")
    print("3. Verify AkShare is updated to the latest version")
    print("4. Check if the market is supported by AkShare")

Attempting to fetch data for 588000 from 20200723 to 20250722

Error: Could not fetch data for 588000. using AkShare
Reason: No data returned from AkShare

Possible solutions:
1. Check if the symbol is correct and exists
2. Try alternative data sources (Yahoo Finance, Tushare, etc.)
3. Verify AkShare is updated to the latest version
4. Check if the market is supported by AkShare


In [1]:
import akshare as ak
fund_em_info_df = ak.fund_em_open_fund_info(fund="710001", indicator="单位净值走势")
print(fund_em_info_df)

AttributeError: module 'akshare' has no attribute 'fund_em_open_fund_info'

In [5]:
import akshare as ak

# Method 1: Print version directly
print("AkShare version:", ak.__version__)

# Method 2: Get version as a variable
akshare_version = ak.__version__
print(f"Current AkShare version: {akshare_version}")

AkShare version: 1.17.25
Current AkShare version: 1.17.25


In [14]:
# Option 1: From East Money
etf_hist = ak.fund_etf_fund_info_em(fund="588000",start_date="20210101", end_date="20250801") 
etf_hist

  0%|          | 0/56 [00:00<?, ?it/s]

,净值日期,单位净值,累计净值,日增长率,申购状态,赎回状态
0,2021-01-04,1.4658,1.0180,2.08,场内买入,场内卖出
1,2021-01-05,1.4690,1.0202,0.22,场内买入,场内卖出
2,2021-01-06,1.4587,1.0130,-0.70,场内买入,场内卖出
3,2021-01-07,1.4247,0.9894,-2.33,场内买入,场内卖出
4,2021-01-08,1.4215,0.9872,-0.22,场内买入,场内卖出
...,...,...,...,...,...,...
1100,2025-07-16,1.0492,0.7287,0.13,场内买入,场内卖出
1101,2025-07-17,1.0577,0.7346,0.81,场内买入,场内卖出
1102,2025-07-18,1.0596,0.7359,0.18,场内买入,场内卖出
1103,2025-07-21,1.0602,0.7363,0.06,场内买入,场内卖出


In [6]:
print(ak.__version__) 

1.17.25


In [9]:
import akshare as ak

# 不复权
fund_etf_hist_em_df = ak.fund_etf_hist_em(symbol="159915", period="daily", start_date="20210101", end_date="20240801", adjust="")
# print(fund_etf_hist_em_df)
# 前复权
fund_etf_hist_em_df = ak.fund_etf_hist_em(symbol="159915", period="daily", start_date="20210101", end_date="20280801", adjust="qfq")
# print(fund_etf_hist_em_df)
# 后复权
fund_etf_hist_em_df = ak.fund_etf_hist_em(symbol="159915", period="daily", start_date="20210101", end_date="20240801", adjust="hfq")
fund_etf_hist_em_df

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2021-01-04,2.864,2.976,2.990,2.861,2472764,7.278150e+08,4.50,3.84,0.110,0.65
1,2021-01-05,2.940,2.988,2.997,2.925,2470004,7.346481e+08,2.42,0.40,0.012,0.65
2,2021-01-06,3.000,3.003,3.030,2.953,2290438,6.844461e+08,2.58,0.50,0.015,0.60
3,2021-01-07,2.996,3.056,3.056,2.978,2029990,6.118732e+08,2.60,1.76,0.053,0.53
4,2021-01-08,3.065,3.036,3.086,3.011,1867431,5.666295e+08,2.45,-0.65,-0.020,0.49
...,...,...,...,...,...,...,...,...,...,...,...
862,2024-07-26,1.617,1.627,1.638,1.615,7502581,1.220286e+09,1.43,0.81,0.013,1.97
863,2024-07-29,1.622,1.604,1.623,1.603,5724428,9.203704e+08,1.23,-1.41,-0.023,1.50
864,2024-07-30,1.600,1.601,1.607,1.591,6106035,9.750274e+08,1.00,-0.19,-0.003,1.60
865,2024-07-31,1.594,1.658,1.658,1.592,14320519,2.339589e+09,4.12,3.56,0.057,3.76


In [10]:
import akshare as ak
fund_etf_spot_em_df = ak.fund_etf_spot_em()
fund_etf_spot_simple = fund_etf_spot_em_df[["代码","名称","最新价","涨跌幅","成交量","成交额"]]
fund_etf_spot_simple

  0%|          | 0/11 [00:00<?, ?it/s]

,代码,名称,最新价,涨跌幅,成交量,成交额
0,515220,煤炭ETF,1.129,8.25,14724894.0,1.611466e+09
1,159787,建材ETF易方达,0.764,7.91,3168765.0,2.327759e+08
2,516950,基建ETF,1.225,6.99,5301535.0,6.304792e+08
3,516750,建材ETF,0.761,6.88,13390974.0,9.870806e+08
4,159745,建材ETF,0.695,6.76,19601742.0,1.315050e+09
...,...,...,...,...,...,...
1182,516770,游戏ETF华泰柏瑞,1.309,-1.43,190416.0,2.500437e+07
1183,512800,银行ETF,0.867,-1.48,18632839.0,1.616244e+09
1184,513360,教育ETF,0.531,-1.48,4702457.0,2.496259e+08
1185,513350,标普油气ETF,0.940,-1.98,1916526.0,1.799527e+08
